In [4]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import pandas as pd
import urllib.request
import requests
import os
'''
미션
픽사베이에서 이미지 다운로드
'''
options=Options()
options.add_experimental_option('detach',True)
options.add_argument('--start-maximized')
service=Service(ChromeDriverManager().install())

driver=webdriver.Chrome(service=service, options=options)

# ✨ 기본 검색어 및 url 설정
search = '고슴도치'
url = f'https://pixabay.com/ko/photos/search/{search}/?pagi=1'
# ✨ 이미지 저장시 403에러로 해더 설정
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'
hdr = {
    'User-Agent':user_agent
}

# ✨ url 접속
driver.get(url)
time.sleep(2)

# ✨ 스크롤 내리기
before_location = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(1)
    after_location = driver.execute_script("return document.body.scrollHeight")
    if before_location == after_location:
        break
    before_location = after_location

# ✨ 이미지 선택 및 링크 추출
img_elems=driver.find_elements(By.CSS_SELECTOR,'a.link--WHWzm>img')
img_srcs=[i.get_attribute('src') for i in img_elems if "blank" not in i.get_attribute('src')]

# ✨ 마지막 페이지 번호 조회
max_page_elem = driver.find_element(By.CSS_SELECTOR,'div.position--YADEi')
max_page = int(int(max_page_elem.text.split('/')[1].replace(',',''))/100+1)

# ✨ 2페이지 이후 검색 
for i in range(2,max_page+1):
    new_url = url.replace('pagi=1',f'pagi={i}')
    driver.get(new_url)
    time.sleep(2)
        
    # ✨ 이미지 선택 및 링크 추출
    img_elems=driver.find_elements(By.CSS_SELECTOR,'a.link--WHWzm>img')
    img_tmp=[i.get_attribute('src') for i in img_elems if "blank" not in i.get_attribute('src')]
    img_srcs.extend(img_tmp)

# ✨ 이미지 저장 전 사용자 확인
save_res = input(f"이미지는 {len(img_srcs)}장 저장됩니다. 진행합니까?(y/n)")
if save_res.lower() == 'y':
    # ✨ 저장폴더 만들기
    path = f'./{search}'
    os.makedirs(path, exist_ok=True)

    
    
    # ✨ 이미지 저장
    for idx, i in enumerate(img_srcs):
        req = requests.get(i,headers=hdr)
        with open(f'{path}/{search}{idx:04d}.jpg', "wb") as f:
            f.write(req.content)
            
# ✨ 종료
driver.quit()
print("프로그램을 종료합니다.")

이미지는 199장 저장됩니다. 진행합니까?(y/n) y


프로그램을 종료합니다.
